In [162]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 500
import matplotlib.pyplot as plt
#matplotlib inline
import seaborn as sns
import math
import re

import sklearn

import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from typing import List, Tuple

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

import datetime

In [163]:
# путь расположения выборки
path_train = '/home/tefi/Homework/Sport/assignment_2_train.csv'
path_test = '/home/tefi/Homework/Sport/assignment_2_test.csv'
# загрузка данных из файла
train = pd.read_csv(path_train)
test = pd.read_csv(path_test)

train = train[0:20000]
test = test[0:20000]


train_target = train.copy()
train_target = train['isFraud']                # целевая переменная для тренировочных данных
train_target.head(3)
train_data = train.copy()
train_data = train_data.drop(['isFraud'], axis = 'columns')
test_target = test.copy()
test_target = test['isFraud']                # целевая переменная для тренировочных данных
test_data = test.copy()
test_data = test_data.drop(['isFraud'], axis = 'columns')

In [164]:
f = train_data.select_dtypes(include = ['float64', 'int64']).columns
for i in f:
    train_data.loc[(train_data[i].isnull(), [i])] = train_data[i].median()

cat = train_data.select_dtypes(include = ['object']).columns
for i in cat:
    train_data.loc[(train_data[i].isnull(), [i])] = train_data[i].mode()[0]

In [165]:
f = test_data.select_dtypes(include = ['float64', 'int64']).columns
for i in f:
    test_data.loc[(test_data[i].isnull(), [i])] = test_data[i].median()

cat = test_data.select_dtypes(include = ['object']).columns
for i in cat:
    test_data.loc[(test_data[i].isnull(), [i])] = test_data[i].mode()[0]

In [166]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cat = test_data.select_dtypes(include = ['object']).columns

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)

0:	test: 0.6346841	best: 0.6346841 (0)	total: 47.1ms	remaining: 9.37s
10:	test: 0.7809393	best: 0.7809393 (10)	total: 556ms	remaining: 9.56s
20:	test: 0.8011467	best: 0.8011467 (20)	total: 1.09s	remaining: 9.32s
30:	test: 0.7958378	best: 0.8016443 (21)	total: 1.63s	remaining: 8.86s
40:	test: 0.8210803	best: 0.8210803 (40)	total: 2.12s	remaining: 8.22s
50:	test: 0.8389385	best: 0.8389385 (50)	total: 2.63s	remaining: 7.7s
60:	test: 0.8481135	best: 0.8481135 (60)	total: 3.15s	remaining: 7.19s
70:	test: 0.8513730	best: 0.8513730 (70)	total: 3.67s	remaining: 6.66s
80:	test: 0.8542714	best: 0.8542714 (80)	total: 4.18s	remaining: 6.14s
90:	test: 0.8567930	best: 0.8567930 (90)	total: 4.72s	remaining: 5.65s
100:	test: 0.8584913	best: 0.8584913 (100)	total: 5.34s	remaining: 5.24s
110:	test: 0.8584717	best: 0.8585805 (103)	total: 5.86s	remaining: 4.69s
120:	test: 0.8585455	best: 0.8585805 (103)	total: 6.37s	remaining: 4.16s
130:	test: 0.8587017	best: 0.8587017 (130)	total: 6.87s	remaining: 3.62s


In [167]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")



Train-score: 0.586, Valid-score: 0.57, Test-score: 0.539


In [168]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()


0:	total: 56.3ms	remaining: 11.2s
10:	total: 573ms	remaining: 9.85s
20:	total: 1.07s	remaining: 9.14s
30:	total: 1.53s	remaining: 8.36s
40:	total: 2.03s	remaining: 7.86s
50:	total: 2.51s	remaining: 7.34s
60:	total: 3s	remaining: 6.85s
70:	total: 3.49s	remaining: 6.34s
80:	total: 3.97s	remaining: 5.84s
90:	total: 4.47s	remaining: 5.35s
100:	total: 4.97s	remaining: 4.87s
110:	total: 5.55s	remaining: 4.45s
120:	total: 6.04s	remaining: 3.94s
130:	total: 6.48s	remaining: 3.42s
140:	total: 6.96s	remaining: 2.91s
150:	total: 7.46s	remaining: 2.42s
160:	total: 7.91s	remaining: 1.92s
170:	total: 8.41s	remaining: 1.43s
180:	total: 8.91s	remaining: 935ms
190:	total: 9.4s	remaining: 443ms
199:	total: 9.86s	remaining: 0us
0:	total: 47.8ms	remaining: 9.51s
10:	total: 574ms	remaining: 9.86s
20:	total: 1.14s	remaining: 9.71s
30:	total: 1.67s	remaining: 9.1s
40:	total: 2.18s	remaining: 8.46s
50:	total: 2.67s	remaining: 7.81s
60:	total: 3.17s	remaining: 7.22s
70:	total: 3.66s	remaining: 6.65s
80:	total:

0.8465826878098337

# 1

In [169]:
train_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000.0,86400.0,68.5,W,13926.0,360.0,150.0,discover,142.0,credit,315.0,87.0,19.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,115.0,13.0,68.0,13.0,0.0,2.0,56.729166,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,T,T,T,M2,F,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,89.000000,0.0,0.0,0.0,30.0,50.0,100.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2987001.0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,credit,325.0,87.0,9.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,115.0,10.0,0.0,13.0,0.0,2.0,56.729166,0.666666,0.0,63.0,0.0,0.0,0.0,0.0,T,T,T,M0,T,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.

In [170]:
train_data['TransactionDT']

0         86400.0
1         86401.0
2         86469.0
3         86499.0
4         86506.0
           ...   
19995    517180.0
19996    517182.0
19997    517196.0
19998    517201.0
19999    517208.0
Name: TransactionDT, Length: 20000, dtype: float64

In [171]:
day = datetime.datetime(2017, 12, 1)

for i in train_data['TransactionDT']:
    delta = datetime.timedelta(seconds = i)
    future = day + delta
    train_data.loc[train_data['TransactionDT'] == i, ['TransactionDT']] = future

In [172]:
for i in test_data['TransactionDT']:
    delta = datetime.timedelta(seconds = i)
    future = day + delta
    test_data.loc[test_data['TransactionDT'] == i, ['TransactionDT']] = future   

In [173]:
train_data['TransactionDT'] = pd.to_datetime(train_data['TransactionDT'], format='%Y-%m-%d %H:%M:%S')

test_data['TransactionDT'] = pd.to_datetime(test_data['TransactionDT'], format='%Y-%m-%d %H:%M:%S')

In [174]:
train_data['TransactionDT']

0       2017-12-02 00:00:00
1       2017-12-02 00:00:01
2       2017-12-02 00:01:09
3       2017-12-02 00:01:39
4       2017-12-02 00:01:46
                ...        
19995   2017-12-06 23:39:40
19996   2017-12-06 23:39:42
19997   2017-12-06 23:39:56
19998   2017-12-06 23:40:01
19999   2017-12-06 23:40:08
Name: TransactionDT, Length: 20000, dtype: datetime64[ns]

In [175]:
train_data['year'] = train_data['TransactionDT'].dt.year
train_data['month'] = train_data['TransactionDT'].dt.month
train_data['day'] = train_data['TransactionDT'].dt.day
train_data['weekday'] = train_data['TransactionDT'].dt.weekday
train_data['hour'] = train_data['TransactionDT'].dt.hour

test_data['year'] = test_data['TransactionDT'].dt.year
test_data['month'] = test_data['TransactionDT'].dt.month
test_data['day'] = test_data['TransactionDT'].dt.day
test_data['weekday'] = test_data['TransactionDT'].dt.weekday
test_data['hour'] = test_data['TransactionDT'].dt.hour

In [176]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)



0:	test: 0.5413664	best: 0.5413664 (0)	total: 49.7ms	remaining: 9.9s
10:	test: 0.7543997	best: 0.7554734 (6)	total: 542ms	remaining: 9.31s
20:	test: 0.7673877	best: 0.7687059 (16)	total: 1.08s	remaining: 9.18s
30:	test: 0.7790014	best: 0.7790014 (30)	total: 1.71s	remaining: 9.34s
40:	test: 0.8169805	best: 0.8169927 (39)	total: 2.22s	remaining: 8.61s
50:	test: 0.8369873	best: 0.8369873 (50)	total: 2.79s	remaining: 8.16s
60:	test: 0.8439082	best: 0.8439082 (60)	total: 3.35s	remaining: 7.63s
70:	test: 0.8477494	best: 0.8480253 (69)	total: 3.94s	remaining: 7.16s
80:	test: 0.8506306	best: 0.8506306 (80)	total: 4.47s	remaining: 6.57s
90:	test: 0.8516805	best: 0.8521024 (89)	total: 5s	remaining: 5.98s
100:	test: 0.8522244	best: 0.8524666 (96)	total: 5.57s	remaining: 5.46s
110:	test: 0.8519560	best: 0.8527265 (102)	total: 6.11s	remaining: 4.89s
120:	test: 0.8526647	best: 0.8529316 (113)	total: 6.64s	remaining: 4.33s
130:	test: 0.8527690	best: 0.8529900 (124)	total: 7.15s	remaining: 3.77s
140:	

In [177]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")


Train-score: 0.579, Valid-score: 0.608, Test-score: 0.555


In [178]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 46.3ms	remaining: 9.21s
10:	total: 537ms	remaining: 9.22s
20:	total: 1.03s	remaining: 8.78s
30:	total: 1.52s	remaining: 8.28s
40:	total: 1.99s	remaining: 7.73s
50:	total: 2.52s	remaining: 7.35s
60:	total: 2.99s	remaining: 6.81s
70:	total: 3.5s	remaining: 6.37s
80:	total: 3.99s	remaining: 5.86s
90:	total: 4.5s	remaining: 5.38s
100:	total: 4.97s	remaining: 4.87s
110:	total: 5.51s	remaining: 4.42s
120:	total: 5.98s	remaining: 3.9s
130:	total: 6.47s	remaining: 3.41s
140:	total: 6.93s	remaining: 2.9s
150:	total: 7.4s	remaining: 2.4s
160:	total: 7.91s	remaining: 1.92s
170:	total: 8.42s	remaining: 1.43s
180:	total: 8.91s	remaining: 936ms
190:	total: 9.39s	remaining: 443ms
199:	total: 9.85s	remaining: 0us
0:	total: 55.5ms	remaining: 11s
10:	total: 571ms	remaining: 9.82s
20:	total: 1.1s	remaining: 9.36s
30:	total: 1.57s	remaining: 8.56s
40:	total: 2.07s	remaining: 8.02s
50:	total: 2.55s	remaining: 7.45s
60:	total: 3.07s	remaining: 7s
70:	total: 3.59s	remaining: 6.52s
80:	total: 4.1s	r

0.8447198371705236

# 2

In [179]:
train_data.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,weekday,hour
0,2987000.0,2017-12-02 00:00:00,68.5,W,13926.0,360.0,150.0,discover,142.0,credit,315.0,87.0,19.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,115.0,13.0,68.0,13.0,0.0,2.0,56.729166,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,T,T,T,M2,F,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,89.000000,0.0,0.0,0.0,30.0,50.0,100.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,12,2,5,0
1,2987001.0,2017-12-02 00:00:01,29.0,W,2755.0,404.0,150.0,mastercard,102.0,credit,325.0,87.0,9.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,115.0,10.0,0.0,13.0,0.0,2.0,56.729166,0.666666,0.0,63.0,0.0,0.0,0.0,0.0,T,T,T,M0,T,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,

In [180]:
train_data['card1+2'] = train_data['card1'] + train_data['card2']
test_data['card1+2'] = test_data['card1'] + test_data['card2']

In [181]:
train_data['card1+2+3+5'] = train_data['card1'] + train_data['card2'] + train_data['card3'] + train_data['card5']
test_data['card1+2+3+5'] = test_data['card1'] + test_data['card2'] + test_data['card3'] + test_data['card5']

In [182]:
train_data['card1+2+3+5+addr'] = train_data['card1'] + train_data['card2'] + train_data['card3'] + train_data['card5'] + train_data['addr1'] + train_data['addr2']
test_data['card1+2+3+5+addr'] = test_data['card1'] + test_data['card2'] + test_data['card3'] + test_data['card5'] + test_data['addr1'] + test_data['addr2']

In [183]:
train_data.dtypes

TransactionID              float64
TransactionDT       datetime64[ns]
TransactionAmt             float64
ProductCD                   object
card1                      float64
                         ...      
weekday                      int64
hour                         int64
card1+2                    float64
card1+2+3+5                float64
card1+2+3+5+addr           float64
Length: 401, dtype: object

In [184]:
train_data["card1+2"] = train_data["card1+2"].astype(int).astype(object)
train_data["card1+2+3+5"] = train_data["card1+2+3+5"].astype(int).astype(object)
train_data["card1+2+3+5+addr"] = train_data["card1+2+3+5+addr"].astype(int).astype(object)


cat = train_data.select_dtypes(include = ['object']).columns
cat

Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'card1+2',
       'card1+2+3+5', 'card1+2+3+5+addr'],
      dtype='object')

In [185]:
test_data["card1+2"] = test_data["card1+2"].astype(int).astype(object)
test_data["card1+2+3+5"] = test_data["card1+2+3+5"].astype(int).astype(object)
test_data["card1+2+3+5+addr"] = test_data["card1+2+3+5+addr"].astype(int).astype(object)


cat = test_data.select_dtypes(include = ['object']).columns
cat

Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'card1+2',
       'card1+2+3+5', 'card1+2+3+5+addr'],
      dtype='object')

In [186]:
test_data['card1+2'].isnull().sum()

0

In [187]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 401 entries, TransactionID to card1+2+3+5+addr
dtypes: datetime64[ns](1), float64(378), int64(5), object(17)
memory usage: 61.2+ MB


In [188]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 401 entries, TransactionID to card1+2+3+5+addr
dtypes: datetime64[ns](1), float64(378), int64(5), object(17)
memory usage: 61.2+ MB


In [189]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)



0:	test: 0.5664524	best: 0.5664524 (0)	total: 51.7ms	remaining: 10.3s
10:	test: 0.6792871	best: 0.6792871 (10)	total: 635ms	remaining: 10.9s
20:	test: 0.7813717	best: 0.7813717 (20)	total: 1.2s	remaining: 10.2s
30:	test: 0.7980542	best: 0.7981335 (26)	total: 1.8s	remaining: 9.83s
40:	test: 0.8196306	best: 0.8196306 (40)	total: 2.39s	remaining: 9.26s
50:	test: 0.9286037	best: 0.9286037 (50)	total: 2.96s	remaining: 8.65s
60:	test: 0.9346729	best: 0.9346729 (60)	total: 3.52s	remaining: 8.02s
70:	test: 0.9416850	best: 0.9416850 (70)	total: 4.14s	remaining: 7.52s
80:	test: 0.9442102	best: 0.9442102 (80)	total: 4.75s	remaining: 6.98s
90:	test: 0.9455214	best: 0.9455214 (90)	total: 5.33s	remaining: 6.38s
100:	test: 0.9478839	best: 0.9479131 (99)	total: 5.91s	remaining: 5.79s
110:	test: 0.9492416	best: 0.9492638 (108)	total: 6.49s	remaining: 5.2s
120:	test: 0.9533566	best: 0.9533566 (120)	total: 7.05s	remaining: 4.6s
130:	test: 0.9565427	best: 0.9565935 (129)	total: 7.59s	remaining: 4s
140:	te

In [190]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")



Train-score: 0.71, Valid-score: 0.668, Test-score: 0.54


In [191]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 61.1ms	remaining: 12.2s
10:	total: 639ms	remaining: 11s
20:	total: 1.15s	remaining: 9.83s
30:	total: 1.74s	remaining: 9.47s
40:	total: 2.28s	remaining: 8.86s
50:	total: 2.89s	remaining: 8.43s
60:	total: 3.46s	remaining: 7.88s
70:	total: 4.05s	remaining: 7.35s
80:	total: 4.62s	remaining: 6.79s
90:	total: 5.21s	remaining: 6.25s
100:	total: 5.75s	remaining: 5.63s
110:	total: 6.3s	remaining: 5.05s
120:	total: 6.84s	remaining: 4.47s
130:	total: 7.42s	remaining: 3.91s
140:	total: 7.98s	remaining: 3.34s
150:	total: 8.61s	remaining: 2.79s
160:	total: 9.21s	remaining: 2.23s
170:	total: 9.79s	remaining: 1.66s
180:	total: 10.4s	remaining: 1.09s
190:	total: 11s	remaining: 519ms
199:	total: 11.5s	remaining: 0us
0:	total: 54.3ms	remaining: 10.8s
10:	total: 619ms	remaining: 10.6s
20:	total: 1.15s	remaining: 9.84s
30:	total: 1.71s	remaining: 9.32s
40:	total: 2.34s	remaining: 9.07s
50:	total: 2.91s	remaining: 8.5s
60:	total: 3.47s	remaining: 7.9s
70:	total: 4.04s	remaining: 7.33s
80:	total: 4

0.8588719700327514

# 3

In [192]:
# FrequencyEncoder

encoding = train_data.groupby('card1').size()
encoding = encoding/len(train_data)
train_data['enc_card1'] = train_data.card1.map(encoding)

encoding = train_data.groupby('card2').size()
encoding = encoding/len(train_data)
train_data['enc_card2'] = train_data.card2.map(encoding)

encoding = train_data.groupby('card3').size()
encoding = encoding/len(train_data)
train_data['enc_card3'] = train_data.card3.map(encoding)

encoding = train_data.groupby('card4').size()
encoding = encoding/len(train_data)
train_data['enc_card4'] = train_data.card4.map(encoding)

encoding = train_data.groupby('card5').size()
encoding = encoding/len(train_data)
train_data['enc_card5'] = train_data.card5.map(encoding)

encoding = train_data.groupby('card6').size()
encoding = encoding/len(train_data)
train_data['enc_card6'] = train_data.card6.map(encoding)

encoding = train_data.groupby('addr1').size()
encoding = encoding/len(train_data)
train_data['enc_addr1'] = train_data.addr1.map(encoding)

encoding = train_data.groupby('addr2').size()
encoding = encoding/len(train_data)
train_data['enc_addr2'] = train_data.addr2.map(encoding)

In [193]:
encoding = test_data.groupby('card1').size()
encoding = encoding/len(test_data)
test_data['enc_card1'] = test_data.card1.map(encoding)

encoding = test_data.groupby('card2').size()
encoding = encoding/len(test_data)
test_data['enc_card2'] = test_data.card2.map(encoding)

encoding = test_data.groupby('card3').size()
encoding = encoding/len(test_data)
test_data['enc_card3'] = test_data.card3.map(encoding)

encoding = test_data.groupby('card4').size()
encoding = encoding/len(test_data)
test_data['enc_card4'] = test_data.card4.map(encoding)

encoding = test_data.groupby('card5').size()
encoding = encoding/len(test_data)
test_data['enc_card5'] = test_data.card5.map(encoding)

encoding = test_data.groupby('card6').size()
encoding = encoding/len(test_data)
test_data['enc_card6'] = test_data.card6.map(encoding)

encoding = test_data.groupby('addr1').size()
encoding = encoding/len(test_data)
test_data['enc_addr1'] = test_data.addr1.map(encoding)

encoding = test_data.groupby('addr2').size()
encoding = encoding/len(test_data)
test_data['enc_addr2'] = test_data.addr2.map(encoding)

In [194]:
train_data

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,weekday,hour,card1+2,card1+2+3+5,card1+2+3+5+addr,enc_card1,enc_card2,enc_card3,enc_card4,enc_card5,enc_card6,enc_addr1,enc_addr2
0,2987000.0,2017-12-02 00:00:00,68.500,W,13926.0,360.0,150.0,discover,142.0,credit,315.0,87.0,19.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,115.0,13.0,68.0,13.0,0.0,2.0,56.729166,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,T,T,T,M2,F,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,89.000000,0.0,0.0,0.0,30.0,50.0,100.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0000,117.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,12,2,5,0,14286,14578,14980,0.00005,0.03160,0.90780,0.01110,0.00035,0.25340,0.04600,0.98995
1,2987001.0,2017-12-02 00:00:01,29.000,W,2755.0,404.0,150.0,mastercard,102.0,credit,325.0,87.0,9.0,24.5,gmail.com,g

In [195]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)



0:	test: 0.5641869	best: 0.5641869 (0)	total: 56.1ms	remaining: 11.2s
10:	test: 0.7948600	best: 0.7948600 (10)	total: 652ms	remaining: 11.2s
20:	test: 0.8070832	best: 0.8071126 (19)	total: 1.27s	remaining: 10.8s
30:	test: 0.8263958	best: 0.8263958 (30)	total: 1.92s	remaining: 10.5s
40:	test: 0.8346494	best: 0.8346494 (40)	total: 2.53s	remaining: 9.81s
50:	test: 0.9134548	best: 0.9134548 (50)	total: 3.13s	remaining: 9.15s
60:	test: 0.9377545	best: 0.9377545 (60)	total: 3.71s	remaining: 8.46s
70:	test: 0.9438275	best: 0.9438275 (70)	total: 4.32s	remaining: 7.85s
80:	test: 0.9464769	best: 0.9464769 (80)	total: 4.98s	remaining: 7.32s
90:	test: 0.9534878	best: 0.9534878 (90)	total: 5.57s	remaining: 6.67s
100:	test: 0.9575067	best: 0.9575067 (100)	total: 6.16s	remaining: 6.04s
110:	test: 0.9581026	best: 0.9581107 (109)	total: 6.74s	remaining: 5.41s
120:	test: 0.9587766	best: 0.9587766 (120)	total: 7.35s	remaining: 4.8s
130:	test: 0.9592772	best: 0.9592772 (130)	total: 7.95s	remaining: 4.19s


In [196]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")



Train-score: 0.717, Valid-score: 0.661, Test-score: 0.539


In [197]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 58.2ms	remaining: 11.6s
10:	total: 594ms	remaining: 10.2s
20:	total: 1.19s	remaining: 10.1s
30:	total: 1.81s	remaining: 9.87s
40:	total: 2.41s	remaining: 9.36s
50:	total: 3.03s	remaining: 8.86s
60:	total: 3.67s	remaining: 8.35s
70:	total: 4.22s	remaining: 7.66s
80:	total: 4.77s	remaining: 7.01s
90:	total: 5.34s	remaining: 6.39s
100:	total: 5.89s	remaining: 5.78s
110:	total: 6.46s	remaining: 5.18s
120:	total: 7.01s	remaining: 4.58s
130:	total: 7.58s	remaining: 3.99s
140:	total: 8.18s	remaining: 3.42s
150:	total: 8.86s	remaining: 2.88s
160:	total: 9.49s	remaining: 2.3s
170:	total: 10.1s	remaining: 1.71s
180:	total: 10.7s	remaining: 1.12s
190:	total: 11.3s	remaining: 533ms
199:	total: 11.8s	remaining: 0us
0:	total: 58.7ms	remaining: 11.7s
10:	total: 631ms	remaining: 10.8s
20:	total: 1.2s	remaining: 10.2s
30:	total: 1.76s	remaining: 9.62s
40:	total: 2.46s	remaining: 9.53s
50:	total: 3.03s	remaining: 8.85s
60:	total: 3.57s	remaining: 8.14s
70:	total: 4.1s	remaining: 7.45s
80:	tota

0.862200927141834

# 4

In [198]:
#dr = train_data.groupby("card1")['TransactionAmt'].mean()
#for i in dr.index:

#train_data.loc[train_data['card1'] == i, 'gr_c1'] = dr[i]

In [199]:
train_data['gr_c1_m'] = train_data.groupby(['card1'])['TransactionAmt'].transform('mean')
train_data['gr_c1_m'] = train_data['TransactionAmt'] / train_data['gr_c1_m']

train_data['gr_c1_s'] = train_data.groupby(['card1'])['TransactionAmt'].transform('std')
train_data['gr_c1_s'] = train_data['TransactionAmt'] / train_data['gr_c1_s']

train_data['gr_c2_m'] = train_data.groupby(['card2'])['TransactionAmt'].transform('mean')
train_data['gr_c2_m'] = train_data['TransactionAmt'] / train_data['gr_c2_m']

train_data['gr_c2_s'] = train_data.groupby(['card2'])['TransactionAmt'].transform('std')
train_data['gr_c2_s'] = train_data['TransactionAmt'] / train_data['gr_c2_s']

train_data['gr_c3_m'] = train_data.groupby(['card3'])['TransactionAmt'].transform('mean')
train_data['gr_c3_m'] = train_data['TransactionAmt'] / train_data['gr_c3_m']

train_data['gr_c3_s'] = train_data.groupby(['card3'])['TransactionAmt'].transform('std')
train_data['gr_c3_s'] = train_data['TransactionAmt'] / train_data['gr_c3_s']

train_data['gr_c4_m'] = train_data.groupby(['card4'])['TransactionAmt'].transform('mean')
train_data['gr_c4_m'] = train_data['TransactionAmt'] / train_data['gr_c4_m']

train_data['gr_c4_s'] = train_data.groupby(['card4'])['TransactionAmt'].transform('std')
train_data['gr_c4_s'] = train_data['TransactionAmt'] / train_data['gr_c4_s']

train_data['gr_c5_m'] = train_data.groupby(['card5'])['TransactionAmt'].transform('mean')
train_data['gr_c5_m'] = train_data['TransactionAmt'] / train_data['gr_c5_m']

train_data['gr_c5_s'] = train_data.groupby(['card5'])['TransactionAmt'].transform('std')
train_data['gr_c5_s'] = train_data['TransactionAmt'] / train_data['gr_c5_s']

train_data['gr_c6_m'] = train_data.groupby(['card6'])['TransactionAmt'].transform('mean')
train_data['gr_c6_m'] = train_data['TransactionAmt'] / train_data['gr_c6_m']

train_data['gr_c6_s'] = train_data.groupby(['card6'])['TransactionAmt'].transform('std')
train_data['gr_c6_s'] = train_data['TransactionAmt'] / train_data['gr_c6_s']

train_data['gr_a1_m'] = train_data.groupby(['addr1'])['TransactionAmt'].transform('mean')
train_data['gr_a1_m'] = train_data['TransactionAmt'] / train_data['gr_a1_m']

train_data['gr_a1_s'] = train_data.groupby(['addr1'])['TransactionAmt'].transform('std')
train_data['gr_a1_s'] = train_data['TransactionAmt'] / train_data['gr_a1_s']

train_data['gr_a2_m'] = train_data.groupby(['addr2'])['TransactionAmt'].transform('mean')
train_data['gr_a2_m'] = train_data['TransactionAmt'] / train_data['gr_a2_m']

train_data['gr_a2_s'] = train_data.groupby(['addr2'])['TransactionAmt'].transform('std')
train_data['gr_a2_s'] = train_data['TransactionAmt'] / train_data['gr_a2_s']

train_data['gr_c12_m'] = train_data.groupby(['card1+2'])['TransactionAmt'].transform('mean')
train_data['gr_c12_m'] = train_data['TransactionAmt'] / train_data['gr_c12_m']

train_data['gr_c12_s'] = train_data.groupby(['card1+2'])['TransactionAmt'].transform('std')
train_data['gr_c12_s'] = train_data['TransactionAmt'] / train_data['gr_c12_s']

train_data['gr_c1235_m'] = train_data.groupby(['card1+2+3+5'])['TransactionAmt'].transform('mean')
train_data['gr_c1235_m'] = train_data['TransactionAmt'] / train_data['gr_c1235_m']

train_data['gr_c1235_s'] = train_data.groupby(['card1+2+3+5'])['TransactionAmt'].transform('std')
train_data['gr_c1235_s'] = train_data['TransactionAmt'] / train_data['gr_c1235_s']

train_data['gr_c1235a_m'] = train_data.groupby(['card1+2+3+5+addr'])['TransactionAmt'].transform('mean')
train_data['gr_c1235a_m'] = train_data['TransactionAmt'] / train_data['gr_c1235a_m']

train_data['gr_c1235a_s'] = train_data.groupby(['card1+2+3+5+addr'])['TransactionAmt'].transform('std')
train_data['gr_c1235a_s'] = train_data['TransactionAmt'] / train_data['gr_c1235a_s']

In [200]:
test_data['gr_c1_m'] = test_data.groupby(['card1'])['TransactionAmt'].transform('mean')
test_data['gr_c1_m'] = test_data['TransactionAmt'] / test_data['gr_c1_m']

test_data['gr_c1_s'] = test_data.groupby(['card1'])['TransactionAmt'].transform('std')
test_data['gr_c1_s'] = test_data['TransactionAmt'] / test_data['gr_c1_s']

test_data['gr_c2_m'] = test_data.groupby(['card2'])['TransactionAmt'].transform('mean')
test_data['gr_c2_m'] = test_data['TransactionAmt'] / test_data['gr_c2_m']

test_data['gr_c2_s'] = test_data.groupby(['card2'])['TransactionAmt'].transform('std')
test_data['gr_c2_s'] = test_data['TransactionAmt'] / test_data['gr_c2_s']

test_data['gr_c3_m'] = test_data.groupby(['card3'])['TransactionAmt'].transform('mean')
test_data['gr_c3_m'] = test_data['TransactionAmt'] / test_data['gr_c3_m']

test_data['gr_c3_s'] = test_data.groupby(['card3'])['TransactionAmt'].transform('std')
test_data['gr_c3_s'] = test_data['TransactionAmt'] / test_data['gr_c3_s']

test_data['gr_c4_m'] = test_data.groupby(['card4'])['TransactionAmt'].transform('mean')
test_data['gr_c4_m'] = test_data['TransactionAmt'] / test_data['gr_c4_m']

test_data['gr_c4_s'] = test_data.groupby(['card4'])['TransactionAmt'].transform('std')
test_data['gr_c4_s'] = test_data['TransactionAmt'] / test_data['gr_c4_s']

test_data['gr_c5_m'] = test_data.groupby(['card5'])['TransactionAmt'].transform('mean')
test_data['gr_c5_m'] = test_data['TransactionAmt'] / test_data['gr_c5_m']

test_data['gr_c5_s'] = test_data.groupby(['card5'])['TransactionAmt'].transform('std')
test_data['gr_c5_s'] = test_data['TransactionAmt'] / test_data['gr_c5_s']

test_data['gr_c6_m'] = test_data.groupby(['card6'])['TransactionAmt'].transform('mean')
test_data['gr_c6_m'] = test_data['TransactionAmt'] / test_data['gr_c6_m']

test_data['gr_c6_s'] = test_data.groupby(['card6'])['TransactionAmt'].transform('std')
test_data['gr_c6_s'] = test_data['TransactionAmt'] / test_data['gr_c6_s']

test_data['gr_a1_m'] = test_data.groupby(['addr1'])['TransactionAmt'].transform('mean')
test_data['gr_a1_m'] = test_data['TransactionAmt'] / test_data['gr_a1_m']

test_data['gr_a1_s'] = test_data.groupby(['addr1'])['TransactionAmt'].transform('std')
test_data['gr_a1_s'] = test_data['TransactionAmt'] / test_data['gr_a1_s']

test_data['gr_a2_m'] = test_data.groupby(['addr2'])['TransactionAmt'].transform('mean')
test_data['gr_a2_m'] = test_data['TransactionAmt'] / test_data['gr_a2_m']

test_data['gr_a2_s'] = test_data.groupby(['addr2'])['TransactionAmt'].transform('std')
test_data['gr_a2_s'] = test_data['TransactionAmt'] / test_data['gr_a2_s']

test_data['gr_c12_m'] = test_data.groupby(['card1+2'])['TransactionAmt'].transform('mean')
test_data['gr_c12_m'] = test_data['TransactionAmt'] / test_data['gr_c12_m']

test_data['gr_c12_s'] = test_data.groupby(['card1+2'])['TransactionAmt'].transform('std')
test_data['gr_c12_s'] = test_data['TransactionAmt'] / test_data['gr_c12_s']

test_data['gr_c1235_m'] = test_data.groupby(['card1+2+3+5'])['TransactionAmt'].transform('mean')
test_data['gr_c1235_m'] = test_data['TransactionAmt'] / test_data['gr_c1235_m']

test_data['gr_c1235_s'] = test_data.groupby(['card1+2+3+5'])['TransactionAmt'].transform('std')
test_data['gr_c1235_s'] = test_data['TransactionAmt'] / test_data['gr_c1235_s']

test_data['gr_c1235a_m'] = test_data.groupby(['card1+2+3+5+addr'])['TransactionAmt'].transform('mean')
test_data['gr_c1235a_m'] = test_data['TransactionAmt'] / test_data['gr_c1235a_m']

test_data['gr_c1235a_s'] = test_data.groupby(['card1+2+3+5+addr'])['TransactionAmt'].transform('std')
test_data['gr_c1235a_s'] = test_data['TransactionAmt'] / test_data['gr_c1235a_s']

In [201]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)


0:	test: 0.5564910	best: 0.5564910 (0)	total: 55ms	remaining: 10.9s
10:	test: 0.7746474	best: 0.7746474 (10)	total: 667ms	remaining: 11.5s
20:	test: 0.7942095	best: 0.7942499 (19)	total: 1.27s	remaining: 10.8s
30:	test: 0.8115538	best: 0.8116220 (29)	total: 1.88s	remaining: 10.2s
40:	test: 0.8199129	best: 0.8201538 (38)	total: 2.51s	remaining: 9.75s
50:	test: 0.8709153	best: 0.8709153 (50)	total: 3.1s	remaining: 9.04s
60:	test: 0.9259692	best: 0.9259692 (60)	total: 3.75s	remaining: 8.54s
70:	test: 0.9377047	best: 0.9377047 (70)	total: 4.38s	remaining: 7.95s
80:	test: 0.9443384	best: 0.9443384 (80)	total: 4.95s	remaining: 7.27s
90:	test: 0.9443298	best: 0.9444964 (88)	total: 5.55s	remaining: 6.64s
100:	test: 0.9459689	best: 0.9459689 (100)	total: 6.13s	remaining: 6.01s
110:	test: 0.9455581	best: 0.9460727 (106)	total: 6.72s	remaining: 5.39s
120:	test: 0.9460594	best: 0.9460727 (106)	total: 7.33s	remaining: 4.78s
130:	test: 0.9470862	best: 0.9470862 (130)	total: 7.92s	remaining: 4.17s
14

In [202]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")


Train-score: 0.693, Valid-score: 0.66, Test-score: 0.54


In [203]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 57.9ms	remaining: 11.5s
10:	total: 611ms	remaining: 10.5s
20:	total: 1.2s	remaining: 10.2s
30:	total: 1.79s	remaining: 9.77s
40:	total: 2.46s	remaining: 9.53s
50:	total: 3.05s	remaining: 8.9s
60:	total: 3.59s	remaining: 8.18s
70:	total: 4.14s	remaining: 7.53s
80:	total: 4.74s	remaining: 6.96s
90:	total: 5.29s	remaining: 6.33s
100:	total: 5.84s	remaining: 5.72s
110:	total: 6.41s	remaining: 5.14s
120:	total: 6.98s	remaining: 4.56s
130:	total: 7.53s	remaining: 3.97s
140:	total: 8.25s	remaining: 3.45s
150:	total: 8.87s	remaining: 2.88s
160:	total: 9.47s	remaining: 2.29s
170:	total: 10.1s	remaining: 1.71s
180:	total: 10.7s	remaining: 1.12s
190:	total: 11.3s	remaining: 533ms
199:	total: 11.9s	remaining: 0us
0:	total: 61.8ms	remaining: 12.3s
10:	total: 605ms	remaining: 10.4s
20:	total: 1.22s	remaining: 10.4s
30:	total: 1.92s	remaining: 10.5s
40:	total: 2.52s	remaining: 9.79s
50:	total: 3.08s	remaining: 9.01s
60:	total: 3.67s	remaining: 8.37s
70:	total: 4.24s	remaining: 7.71s
80:	tot

0.8660339219332183

# 5

In [204]:
train_data['gr_c1_dm'] = train_data.groupby(['card1'])['D15'].transform('mean')
train_data['gr_c1_dm'] = train_data['D15'] / train_data['gr_c1_m']

train_data['gr_c1_ds'] = train_data.groupby(['card1'])['D15'].transform('std')
train_data['gr_c1_ds'] = train_data['D15'] / train_data['gr_c1_s']

train_data['gr_c2_dm'] = train_data.groupby(['card2'])['D15'].transform('mean')
train_data['gr_c2_dm'] = train_data['D15'] / train_data['gr_c2_m']

train_data['gr_c2_ds'] = train_data.groupby(['card2'])['D15'].transform('std')
train_data['gr_c2_ds'] = train_data['D15'] / train_data['gr_c2_s']

train_data['gr_c3_dm'] = train_data.groupby(['card3'])['D15'].transform('mean')
train_data['gr_c3_dm'] = train_data['D15'] / train_data['gr_c3_m']

train_data['gr_c3_ds'] = train_data.groupby(['card3'])['D15'].transform('std')
train_data['gr_c3_ds'] = train_data['D15'] / train_data['gr_c3_s']

train_data['gr_c4_dm'] = train_data.groupby(['card4'])['D15'].transform('mean')
train_data['gr_c4_dm'] = train_data['D15'] / train_data['gr_c4_m']

train_data['gr_c4_ds'] = train_data.groupby(['card4'])['D15'].transform('std')
train_data['gr_c4_ds'] = train_data['D15'] / train_data['gr_c4_s']

train_data['gr_c5_dm'] = train_data.groupby(['card5'])['D15'].transform('mean')
train_data['gr_c5_dm'] = train_data['D15'] / train_data['gr_c5_m']

train_data['gr_c5_ds'] = train_data.groupby(['card5'])['D15'].transform('std')
train_data['gr_c5_ds'] = train_data['D15'] / train_data['gr_c5_s']

train_data['gr_c6_dm'] = train_data.groupby(['card6'])['D15'].transform('mean')
train_data['gr_c6_dm'] = train_data['D15'] / train_data['gr_c6_m']

train_data['gr_c6_ds'] = train_data.groupby(['card6'])['D15'].transform('std')
train_data['gr_c6_ds'] = train_data['D15'] / train_data['gr_c6_s']

train_data['gr_a1_dm'] = train_data.groupby(['addr1'])['D15'].transform('mean')
train_data['gr_a1_dm'] = train_data['D15'] / train_data['gr_a1_m']

train_data['gr_a1_ds'] = train_data.groupby(['addr1'])['D15'].transform('std')
train_data['gr_a1_ds'] = train_data['D15'] / train_data['gr_a1_s']

train_data['gr_a2_dm'] = train_data.groupby(['addr2'])['D15'].transform('mean')
train_data['gr_a2_dm'] = train_data['D15'] / train_data['gr_a2_m']

train_data['gr_a2_ds'] = train_data.groupby(['addr2'])['D15'].transform('std')
train_data['gr_a2_ds'] = train_data['D15'] / train_data['gr_a2_s']

train_data['gr_c12_dm'] = train_data.groupby(['card1+2'])['D15'].transform('mean')
train_data['gr_c12_dm'] = train_data['D15'] / train_data['gr_c12_m']

train_data['gr_c12_ds'] = train_data.groupby(['card1+2'])['D15'].transform('std')
train_data['gr_c12_ds'] = train_data['D15'] / train_data['gr_c12_s']

train_data['gr_c1235_dm'] = train_data.groupby(['card1+2+3+5'])['D15'].transform('mean')
train_data['gr_c1235_dm'] = train_data['D15'] / train_data['gr_c1235_m']

train_data['gr_c1235_ds'] = train_data.groupby(['card1+2+3+5'])['D15'].transform('std')
train_data['gr_c1235_ds'] = train_data['D15'] / train_data['gr_c1235_s']

train_data['gr_c1235a_dm'] = train_data.groupby(['card1+2+3+5+addr'])['D15'].transform('mean')
train_data['gr_c1235a_dm'] = train_data['D15'] / train_data['gr_c1235a_m']

train_data['gr_c1235a_ds'] = train_data.groupby(['card1+2+3+5+addr'])['D15'].transform('std')
train_data['gr_c1235a_ds'] = train_data['D15'] / train_data['gr_c1235a_s']

In [205]:
test_data['gr_c1_dm'] = test_data.groupby(['card1'])['D15'].transform('mean')
test_data['gr_c1_dm'] = test_data['D15'] / test_data['gr_c1_m']

test_data['gr_c1_ds'] = test_data.groupby(['card1'])['D15'].transform('std')
test_data['gr_c1_ds'] = test_data['D15'] / test_data['gr_c1_s']

test_data['gr_c2_dm'] = test_data.groupby(['card2'])['D15'].transform('mean')
test_data['gr_c2_dm'] = test_data['D15'] / test_data['gr_c2_m']

test_data['gr_c2_ds'] = test_data.groupby(['card2'])['D15'].transform('std')
test_data['gr_c2_ds'] = test_data['D15'] / test_data['gr_c2_s']

test_data['gr_c3_dm'] = test_data.groupby(['card3'])['D15'].transform('mean')
test_data['gr_c3_dm'] = test_data['D15'] / test_data['gr_c3_m']

test_data['gr_c3_ds'] = test_data.groupby(['card3'])['D15'].transform('std')
test_data['gr_c3_ds'] = test_data['D15'] / test_data['gr_c3_s']

test_data['gr_c4_dm'] = test_data.groupby(['card4'])['D15'].transform('mean')
test_data['gr_c4_dm'] = test_data['D15'] / test_data['gr_c4_m']

test_data['gr_c4_ds'] = test_data.groupby(['card4'])['D15'].transform('std')
test_data['gr_c4_ds'] = test_data['D15'] / test_data['gr_c4_s']

test_data['gr_c5_dm'] = test_data.groupby(['card5'])['D15'].transform('mean')
test_data['gr_c5_dm'] = test_data['D15'] / test_data['gr_c5_m']

test_data['gr_c5_ds'] = test_data.groupby(['card5'])['D15'].transform('std')
test_data['gr_c5_ds'] = test_data['D15'] / test_data['gr_c5_s']

test_data['gr_c6_dm'] = test_data.groupby(['card6'])['D15'].transform('mean')
test_data['gr_c6_dm'] = test_data['D15'] / test_data['gr_c6_m']

test_data['gr_c6_ds'] = test_data.groupby(['card6'])['D15'].transform('std')
test_data['gr_c6_ds'] = test_data['D15'] / test_data['gr_c6_s']

test_data['gr_a1_dm'] = test_data.groupby(['addr1'])['D15'].transform('mean')
test_data['gr_a1_dm'] = test_data['D15'] / test_data['gr_a1_m']

test_data['gr_a1_ds'] = test_data.groupby(['addr1'])['D15'].transform('std')
test_data['gr_a1_ds'] = test_data['D15'] / test_data['gr_a1_s']

test_data['gr_a2_dm'] = test_data.groupby(['addr2'])['D15'].transform('mean')
test_data['gr_a2_dm'] = test_data['D15'] / test_data['gr_a2_m']

test_data['gr_a2_ds'] = test_data.groupby(['addr2'])['D15'].transform('std')
test_data['gr_a2_ds'] = test_data['D15'] / test_data['gr_a2_s']

test_data['gr_c12_dm'] = test_data.groupby(['card1+2'])['D15'].transform('mean')
test_data['gr_c12_dm'] = test_data['D15'] / test_data['gr_c12_m']

test_data['gr_c12_ds'] = test_data.groupby(['card1+2'])['D15'].transform('std')
test_data['gr_c12_ds'] = test_data['D15'] / test_data['gr_c12_s']

test_data['gr_c1235_dm'] = test_data.groupby(['card1+2+3+5'])['D15'].transform('mean')
test_data['gr_c1235_dm'] = test_data['D15'] / test_data['gr_c1235_m']

test_data['gr_c1235_ds'] = test_data.groupby(['card1+2+3+5'])['D15'].transform('std')
test_data['gr_c1235_ds'] = test_data['D15'] / test_data['gr_c1235_s']

test_data['gr_c1235a_dm'] = test_data.groupby(['card1+2+3+5+addr'])['D15'].transform('mean')
test_data['gr_c1235a_dm'] = test_data['D15'] / test_data['gr_c1235a_m']

test_data['gr_c1235a_ds'] = test_data.groupby(['card1+2+3+5+addr'])['D15'].transform('std')
test_data['gr_c1235a_ds'] = test_data['D15'] / test_data['gr_c1235a_s']

In [206]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)


0:	test: 0.5493386	best: 0.5493386 (0)	total: 57.4ms	remaining: 11.4s
10:	test: 0.7650700	best: 0.7669169 (7)	total: 716ms	remaining: 12.3s
20:	test: 0.7903404	best: 0.7903404 (20)	total: 1.37s	remaining: 11.7s
30:	test: 0.8140278	best: 0.8140278 (30)	total: 2.17s	remaining: 11.8s
40:	test: 0.8173949	best: 0.8178966 (39)	total: 2.8s	remaining: 10.9s
50:	test: 0.9137792	best: 0.9137792 (50)	total: 3.44s	remaining: 10.1s
60:	test: 0.9321378	best: 0.9321378 (60)	total: 4.05s	remaining: 9.23s
70:	test: 0.9371951	best: 0.9378088 (67)	total: 4.64s	remaining: 8.44s
80:	test: 0.9391054	best: 0.9391054 (80)	total: 5.28s	remaining: 7.76s
90:	test: 0.9441495	best: 0.9441495 (90)	total: 5.9s	remaining: 7.06s
100:	test: 0.9529494	best: 0.9529924 (99)	total: 6.5s	remaining: 6.37s
110:	test: 0.9545624	best: 0.9545799 (109)	total: 7.14s	remaining: 5.72s
120:	test: 0.9566401	best: 0.9566584 (117)	total: 7.91s	remaining: 5.16s
130:	test: 0.9583303	best: 0.9583303 (130)	total: 8.57s	remaining: 4.51s
140:

In [207]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")


Train-score: 0.72, Valid-score: 0.68, Test-score: 0.546


In [208]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 61.1ms	remaining: 12.2s
10:	total: 648ms	remaining: 11.1s
20:	total: 1.27s	remaining: 10.9s
30:	total: 1.87s	remaining: 10.2s
40:	total: 2.48s	remaining: 9.63s
50:	total: 3.07s	remaining: 8.97s
60:	total: 3.63s	remaining: 8.27s
70:	total: 4.2s	remaining: 7.64s
80:	total: 4.87s	remaining: 7.15s
90:	total: 5.55s	remaining: 6.65s
100:	total: 6.13s	remaining: 6.01s
110:	total: 6.72s	remaining: 5.39s
120:	total: 7.3s	remaining: 4.77s
130:	total: 7.87s	remaining: 4.15s
140:	total: 8.43s	remaining: 3.52s
150:	total: 9.04s	remaining: 2.93s
160:	total: 9.68s	remaining: 2.34s
170:	total: 10.3s	remaining: 1.75s
180:	total: 11.1s	remaining: 1.16s
190:	total: 11.8s	remaining: 554ms
199:	total: 12.3s	remaining: 0us
0:	total: 56.1ms	remaining: 11.2s
10:	total: 624ms	remaining: 10.7s
20:	total: 1.16s	remaining: 9.85s
30:	total: 1.76s	remaining: 9.6s
40:	total: 2.36s	remaining: 9.16s
50:	total: 2.95s	remaining: 8.63s
60:	total: 3.56s	remaining: 8.11s
70:	total: 4.23s	remaining: 7.69s
80:	tota

0.8612364114063846

# 6

In [209]:
train_data

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,weekday,hour,card1+2,card1+2+3+5,card1+2+3+5+addr,enc_card1,enc_card2,enc_card3,enc_card4,enc_card5,enc_card6,enc_addr1,enc_addr2,gr_c1_m,gr_c1_s,gr_c2_m,gr_c2_s,gr_c3_m,gr_c3_s,gr_c4_m,gr_c4_s,gr_c5_m,gr_c5_s,gr_c6_m,gr_c6_s,gr_a1_m,gr_a1_s,gr_a2_m,gr_a2_s,gr_c12_m,gr_c12_s,gr_c1235_m,gr_c1235_s,gr_c1235a_m,gr_c1235a_s,gr_c1_dm,gr_c1_ds,gr_c2_dm,gr_c2_ds,gr_c3_dm,gr_c3_ds,gr_c4_dm,gr_c4_ds,gr_c5_dm,gr_c5_ds,gr_c6_dm,gr_c6_ds,gr_a1_dm,gr_a1_ds,gr_a2_dm,gr_a2_ds,gr_c12_dm,gr_c12_ds,gr_c1235_dm,gr_c1235_ds,gr_c1235a_dm,gr_c1235a_ds
0,2987000.0,2017-12-02 00:00:00,68.500,W,13926.0,360.0,150.0,discover,142.0,credit,315.0,87.0,19.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,115.0,13.0,68.0,13.0,0.0,2.0,56.729166,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,T,T,T,M2,F,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,89.000000,0.0,0.0,0.0,30.0,50.0,100.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0

In [210]:
train_data['i_TA'] = train_data['TransactionAmt'] // 1
train_data['p_TA'] = train_data['TransactionAmt'] % 1 * 1000

train_data['i_TA'] = train_data['i_TA'].astype(int)
train_data['p_TA'] = train_data['p_TA'].astype(int)

train_data['l_TA'] = np.log(train_data['TransactionAmt'])

test_data['i_TA'] = test_data['TransactionAmt'] // 1
test_data['p_TA'] = test_data['TransactionAmt'] % 1 * 1000

test_data['i_TA'] = test_data['i_TA'].astype(int)
test_data['p_TA'] = test_data['p_TA'].astype(int)

test_data['l_TA'] = np.log(test_data['TransactionAmt'])

In [211]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)


0:	test: 0.5224758	best: 0.5224758 (0)	total: 34.1ms	remaining: 6.79s
10:	test: 0.7483869	best: 0.7557489 (7)	total: 640ms	remaining: 11s
20:	test: 0.7864447	best: 0.7864447 (20)	total: 1.23s	remaining: 10.5s
30:	test: 0.8120745	best: 0.8120745 (30)	total: 1.87s	remaining: 10.2s
40:	test: 0.8300663	best: 0.8309157 (39)	total: 2.46s	remaining: 9.56s
50:	test: 0.9011800	best: 0.9011800 (50)	total: 3.12s	remaining: 9.11s
60:	test: 0.9262865	best: 0.9262865 (60)	total: 3.74s	remaining: 8.53s
70:	test: 0.9286861	best: 0.9286987 (69)	total: 4.44s	remaining: 8.07s
80:	test: 0.9337242	best: 0.9337242 (80)	total: 5.08s	remaining: 7.47s
90:	test: 0.9345177	best: 0.9345404 (89)	total: 5.71s	remaining: 6.84s
100:	test: 0.9419632	best: 0.9419632 (100)	total: 6.33s	remaining: 6.2s
110:	test: 0.9440827	best: 0.9440853 (108)	total: 6.96s	remaining: 5.58s
120:	test: 0.9477835	best: 0.9477835 (120)	total: 7.57s	remaining: 4.94s
130:	test: 0.9506585	best: 0.9506585 (130)	total: 8.19s	remaining: 4.31s
140

In [212]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")


Train-score: 0.732, Valid-score: 0.68, Test-score: 0.55


In [213]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 61.6ms	remaining: 12.3s
10:	total: 600ms	remaining: 10.3s
20:	total: 1.25s	remaining: 10.7s
30:	total: 1.9s	remaining: 10.4s
40:	total: 2.54s	remaining: 9.83s
50:	total: 3.11s	remaining: 9.1s
60:	total: 3.7s	remaining: 8.43s
70:	total: 4.29s	remaining: 7.8s
80:	total: 4.88s	remaining: 7.18s
90:	total: 5.48s	remaining: 6.56s
100:	total: 6.06s	remaining: 5.94s
110:	total: 6.68s	remaining: 5.36s
120:	total: 7.27s	remaining: 4.75s
130:	total: 7.92s	remaining: 4.17s
140:	total: 8.52s	remaining: 3.57s
150:	total: 9.13s	remaining: 2.96s
160:	total: 9.72s	remaining: 2.35s
170:	total: 10.3s	remaining: 1.75s
180:	total: 10.9s	remaining: 1.15s
190:	total: 11.6s	remaining: 545ms
199:	total: 12.2s	remaining: 0us
0:	total: 61.4ms	remaining: 12.2s
10:	total: 702ms	remaining: 12.1s
20:	total: 1.33s	remaining: 11.4s
30:	total: 1.95s	remaining: 10.6s
40:	total: 2.55s	remaining: 9.89s
50:	total: 3.14s	remaining: 9.18s
60:	total: 3.72s	remaining: 8.47s
70:	total: 4.31s	remaining: 7.84s
80:	total

0.8627327666293384

# 7

In [214]:
train_data['R_emaildomain'].value_counts()

gmail.com           17876
hotmail.com           723
anonymous.com         523
yahoo.com             293
aol.com                97
outlook.com            91
comcast.net            65
yahoo.com.mx           40
icloud.com             32
verizon.net            27
live.com.mx            20
msn.com                19
live.com               17
cox.net                16
sbcglobal.net          14
web.de                 14
hotmail.es             13
me.com                 12
bellsouth.net          10
att.net                10
prodigy.net.mx         10
ymail.com               8
roadrunner.com          6
optonline.net           5
mac.com                 5
servicios-ta.com        4
gmx.de                  4
outlook.es              4
embarqmail.com          4
mail.com                3
q.com                   3
juno.com                3
windstream.net          3
earthlink.net           3
suddenlink.net          2
yahoo.fr                2
twc.com                 2
yahoo.de                2
charter.net 

In [215]:
mail_1 = str(train_data['R_emaildomain'].value_counts().index)
mail_2 = str(train_data['P_emaildomain'].value_counts().index)

In [216]:
m1 = re.findall('\S*yahoo\S*', mail_1)
m1

["'yahoo.com',",
 "'yahoo.com.mx',",
 "'yahoo.fr',",
 "'yahoo.de',",
 "'yahoo.es',",
 "'yahoo.co.uk',"]

In [217]:
m11 = re.findall('\S*yahoo\S*', mail_2)
m11

["'yahoo.com',",
 "'yahoo.com.mx',",
 "'yahoo.de',",
 "'yahoo.fr',",
 "'yahoo.es',",
 "'yahoo.co.uk',"]

In [218]:
m1 = ['yahoo.com','yahoo.com.mx','yahoo.fr','yahoo.de','yahoo.es','yahoo.co.uk']
for i in m1:
    train_data.loc[train_data['R_emaildomain'] == i, 'R_emaildomain'] = 'yahoo.com'
for i in m1:
    train_data.loc[train_data['R_emaildomain'] == i, 'P_emaildomain'] = 'yahoo.com'
    
for i in m1:
    test_data.loc[test_data['R_emaildomain'] == i, 'R_emaildomain'] = 'yahoo.com'
for i in m1:
    test_data.loc[test_data['R_emaildomain'] == i, 'P_emaildomain'] = 'yahoo.com'

In [219]:
m2 = re.findall('\S*hotmail\S*', mail_1)
m2

["'hotmail.com',", "'hotmail.es',", "'hotmail.de',", "'hotmail.fr',"]

In [220]:
m2 = ['hotmail.com','hotmail.es','hotmail.de','hotmail.fr']
for i in m2:
    train_data.loc[train_data['R_emaildomain'] == i, 'R_emaildomain'] = 'hotmail.com'
for i in m2:
    train_data.loc[train_data['R_emaildomain'] == i, 'P_emaildomain'] = 'hotmail.com'
    
for i in m2:
    test_data.loc[test_data['R_emaildomain'] == i, 'R_emaildomain'] = 'hotmail.com'
for i in m2:
    test_data.loc[test_data['R_emaildomain'] == i, 'P_emaildomain'] = 'hotmail.com'

In [221]:
m3 = re.findall('\S*gmail\S*', mail_1)
m3

["Index(['gmail.com',", "'gmail',"]

In [222]:
m3 = ['gmail.com','gmail']
for i in m3:
    train_data.loc[train_data['R_emaildomain'] == i, 'R_emaildomain'] = 'gmail.com'
for i in m3:
    train_data.loc[train_data['R_emaildomain'] == i, 'P_emaildomain'] = 'gmail.com'
    
for i in m3:
    test_data.loc[test_data['R_emaildomain'] == i, 'R_emaildomain'] = 'gmail.com'
for i in m3:
    test_data.loc[test_data['R_emaildomain'] == i, 'P_emaildomain'] = 'gmail.com'

In [223]:
train_data['P_emaildomain'].value_counts()

gmail.com           18153
hotmail.com           741
anonymous.com         383
yahoo.com             362
outlook.com            66
aol.com                61
comcast.net            38
live.com.mx            20
icloud.com             20
verizon.net            15
cox.net                15
msn.com                14
web.de                 14
live.com               11
prodigy.net.mx         10
att.net                 9
sbcglobal.net           9
bellsouth.net           9
me.com                  7
ymail.com               6
juno.com                4
outlook.es              4
servicios-ta.com        4
gmx.de                  4
frontier.com            3
mail.com                3
roadrunner.com          2
earthlink.net           2
q.com                   2
twc.com                 2
windstream.net          2
embarqmail.com          1
cfl.rr.com              1
charter.net             1
live.fr                 1
optonline.net           1
Name: P_emaildomain, dtype: int64

In [224]:
# FrequencyEncoder

encoding = train_data.groupby('R_emaildomain').size()
encoding = encoding/len(train_data)
train_data['enc_R'] = train_data.R_emaildomain.map(encoding)

encoding = train_data.groupby('P_emaildomain').size()
encoding = encoding/len(train_data)
train_data['enc_P'] = train_data.P_emaildomain.map(encoding)

encoding = test_data.groupby('R_emaildomain').size()
encoding = encoding/len(test_data)
test_data['enc_R'] = test_data.R_emaildomain.map(encoding)

encoding = test_data.groupby('P_emaildomain').size()
encoding = encoding/len(test_data)
test_data['enc_P'] = test_data.P_emaildomain.map(encoding)

train_data

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339,year,month,day,weekday,hour,card1+2,card1+2+3+5,card1+2+3+5+addr,enc_card1,enc_card2,enc_card3,enc_card4,enc_card5,enc_card6,enc_addr1,enc_addr2,gr_c1_m,gr_c1_s,gr_c2_m,gr_c2_s,gr_c3_m,gr_c3_s,gr_c4_m,gr_c4_s,gr_c5_m,gr_c5_s,gr_c6_m,gr_c6_s,gr_a1_m,gr_a1_s,gr_a2_m,gr_a2_s,gr_c12_m,gr_c12_s,gr_c1235_m,gr_c1235_s,gr_c1235a_m,gr_c1235a_s,gr_c1_dm,gr_c1_ds,gr_c2_dm,gr_c2_ds,gr_c3_dm,gr_c3_ds,gr_c4_dm,gr_c4_ds,gr_c5_dm,gr_c5_ds,gr_c6_dm,gr_c6_ds,gr_a1_dm,gr_a1_ds,gr_a2_dm,gr_a2_ds,gr_c12_dm,gr_c12_ds,gr_c1235_dm,gr_c1235_ds,gr_c1235a_dm,gr_c1235a_ds,i_TA,p_TA,l_TA,enc_R,enc_P
0,2987000.0,2017-12-02 00:00:00,68.500,W,13926.0,360.0,150.0,discover,142.0,credit,315.0,87.0,19.0,24.5,gmail.com,gmail.com,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,115.0,13.0,68.0,13.0,0.0,2.0,56.729166,0.666666,13.0,13.0,0.0,0.0,0.0,0.0,T,T,T,M2,F,T,F,F,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,89.000000,0.0,0.0,0.0,30.0,50.0,100.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,

In [225]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.30)

cb_params = {
    "n_estimators": 200,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 50,
    "thread_count": 6,
    "random_seed": 42
}

model = cb.CatBoostClassifier(**cb_params)
model.fit(X = x_train, y = y_train, eval_set=[(x_train, y_train)], cat_features=cat)

0:	test: 0.5316075	best: 0.5316075 (0)	total: 63.8ms	remaining: 12.7s
10:	test: 0.7620104	best: 0.7620104 (10)	total: 746ms	remaining: 12.8s
20:	test: 0.7983747	best: 0.7983747 (20)	total: 1.42s	remaining: 12.1s
30:	test: 0.8061096	best: 0.8089029 (25)	total: 2.03s	remaining: 11.1s
40:	test: 0.8214577	best: 0.8214577 (40)	total: 2.64s	remaining: 10.2s
50:	test: 0.9194337	best: 0.9194337 (50)	total: 3.25s	remaining: 9.5s
60:	test: 0.9344317	best: 0.9349200 (58)	total: 3.88s	remaining: 8.85s
70:	test: 0.9390050	best: 0.9399942 (66)	total: 4.52s	remaining: 8.22s
80:	test: 0.9390548	best: 0.9399942 (66)	total: 5.15s	remaining: 7.57s
90:	test: 0.9402421	best: 0.9405968 (83)	total: 5.82s	remaining: 6.97s
100:	test: 0.9424335	best: 0.9425040 (99)	total: 6.5s	remaining: 6.37s
110:	test: 0.9431263	best: 0.9431263 (110)	total: 7.12s	remaining: 5.71s
120:	test: 0.9443552	best: 0.9443552 (120)	total: 7.77s	remaining: 5.08s
130:	test: 0.9446969	best: 0.9446997 (128)	total: 8.44s	remaining: 4.44s
14

In [226]:
train_score = roc_auc_score(y_train, model.predict(x_train))
valid_score = roc_auc_score(y_valid, model.predict(x_valid))
test_score = roc_auc_score(test_target, model.predict(test_data))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")


Train-score: 0.717, Valid-score: 0.637, Test-score: 0.549


In [227]:
model = cb.CatBoostClassifier(**cb_params, cat_features=cat)
scores = cross_val_score(model, train_data, train_target, cv=5, scoring='roc_auc')
scores.mean()

0:	total: 58.8ms	remaining: 11.7s
10:	total: 664ms	remaining: 11.4s
20:	total: 1.32s	remaining: 11.2s
30:	total: 1.92s	remaining: 10.5s
40:	total: 2.51s	remaining: 9.74s
50:	total: 3.13s	remaining: 9.13s
60:	total: 3.79s	remaining: 8.64s
70:	total: 4.36s	remaining: 7.92s
80:	total: 4.96s	remaining: 7.29s
90:	total: 5.57s	remaining: 6.67s
100:	total: 6.18s	remaining: 6.06s
110:	total: 6.79s	remaining: 5.44s
120:	total: 7.4s	remaining: 4.83s
130:	total: 8.01s	remaining: 4.22s
140:	total: 8.62s	remaining: 3.6s
150:	total: 9.27s	remaining: 3.01s
160:	total: 9.89s	remaining: 2.4s
170:	total: 10.5s	remaining: 1.78s
180:	total: 11.1s	remaining: 1.17s
190:	total: 11.8s	remaining: 555ms
199:	total: 12.4s	remaining: 0us
0:	total: 59.3ms	remaining: 11.8s
10:	total: 619ms	remaining: 10.6s
20:	total: 1.2s	remaining: 10.3s
30:	total: 1.84s	remaining: 10s
40:	total: 2.5s	remaining: 9.69s
50:	total: 3.09s	remaining: 9.04s
60:	total: 3.67s	remaining: 8.36s
70:	total: 4.28s	remaining: 7.78s
80:	total: 4

0.8616598927264544